In [1]:
import os
import random
import time
import gc
import copy
import numpy as np
import tensorflow as tf
import kagglehub

from collections import deque
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

2025-12-19 11:36:54.922929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766144214.939664     669 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766144214.944410     669 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766144214.957586     669 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766144214.957625     669 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766144214.957627     669 computation_placer.cc:177] computation placer alr

In [2]:
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


In [3]:
print("Downloading Dataset...")
try:
    path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
    train_dir = os.path.join(path, "Training")
    test_dir  = os.path.join(path, "Testing")
except:
    print("Dataset download failed.")
    train_dir = "Training"
    test_dir  = "Testing"

IMAGE_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    seed=42
)

val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    seed=42
)

NUM_CLASSES = train_gen.num_classes

Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.


In [4]:

IMAGE_SIZE = 224
BATCH_SIZE = 32


In [5]:
CNN_SEARCH_SPACE = {
    "lr": [1e-3, 1e-4, 5e-5],
    "dense_units": [128, 256, 512],
    "dropout": [0.3, 0.4, 0.5]
}

In [6]:
def evaluate_cnn(lr, dropout, dense_units, train_gen, val_gen, epochs=1):
    K.clear_session()

    base = MobileNetV2(
        input_shape=(224,224,3),
        include_top=False,
        weights="imagenet"
    )
    base.trainable = False

    inp = Input(shape=(224,224,3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    x = base(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout)(x)
    x = Dense(dense_units, activation="relu")(x)
    x = Dropout(dropout)(x)
    out = Dense(NUM_CLASSES, activation="softmax")(x)

    model = Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    early_stop = EarlyStopping(
        monitor="val_accuracy",
        patience=1,
        restore_best_weights=True
    )

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stop],
        verbose=0
    )

    acc = max(history.history["val_accuracy"])

    del model
    gc.collect()

    return acc


# Particle swarm

In [7]:

class DiscreteParticle:
    def __init__(self):
        self.pos = {
            "lr": random.uniform(0, 2),
            "dense_units": random.uniform(0, 2),
            "dropout": random.uniform(0, 2)
        }
        self.vel = {k: 0.0 for k in self.pos}
        self.best_pos = self.pos.copy()
        self.best_score = -1

    def get_config(self):
        return {
            "lr": CNN_SEARCH_SPACE["lr"][int(round(self.pos["lr"]))],
            "dense_units": CNN_SEARCH_SPACE["dense_units"][int(round(self.pos["dense_units"]))],
            "dropout": CNN_SEARCH_SPACE["dropout"][int(round(self.pos["dropout"]))],
        }

def run_pso_worker(c1, c2, w, train_gen, val_gen, n_particles=2, n_iters=2):
    print(f"    [Worker PSO] c1={c1}, c2={c2}, w={w}")

    swarm = [DiscreteParticle() for _ in range(n_particles)]
    gbest_score = -1
    gbest_pos = None

    for _ in range(n_iters):
        for p in swarm:
            cfg = p.get_config()
            score = evaluate_cnn(
                cfg["lr"], cfg["dropout"], cfg["dense_units"],
                train_gen, val_gen
            )

            if score > p.best_score:
                p.best_score = score
                p.best_pos = p.pos.copy()

            if score > gbest_score:
                gbest_score = score
                gbest_pos = p.pos.copy()

        for p in swarm:
            for k in p.pos:
                r1, r2 = random.random(), random.random()
                p.vel[k] = (
                    w * p.vel[k]
                    + c1 * r1 * (p.best_pos[k] - p.pos[k])
                    + c2 * r2 * (gbest_pos[k] - p.pos[k])
                )
                p.pos[k] += p.vel[k]
                p.pos[k] = np.clip(p.pos[k], 0, 2)

    return gbest_score


# Stimulated Anealing

In [8]:
def run_sa_worker(initial_temp, cooling_rate, train_gen, val_gen, n_iters=3):
    print(f"    [Worker SA] Temp={initial_temp}, Cooling={cooling_rate}")

    current = {k: random.choice(v) for k, v in CNN_SEARCH_SPACE.items()}
    current_acc = evaluate_cnn(
        current["lr"], current["dropout"], current["dense_units"],
        train_gen, val_gen
    )

    best_acc = current_acc
    temp = initial_temp

    for _ in range(n_iters):
        neighbor = current.copy()
        key = random.choice(list(CNN_SEARCH_SPACE.keys()))
        neighbor[key] = random.choice(CNN_SEARCH_SPACE[key])

        new_acc = evaluate_cnn(
            neighbor["lr"], neighbor["dropout"], neighbor["dense_units"],
            train_gen, val_gen
        )

        if new_acc > current_acc or random.random() < np.exp((new_acc - current_acc)/temp):
            current = neighbor
            current_acc = new_acc

        best_acc = max(best_acc, current_acc)
        temp *= cooling_rate

    return best_acc


# Tabu

In [9]:
def get_neighbors(config, space):
    neighbors = []
    for k, vals in space.items():
        for v in vals:
            if v != config[k]:
                n = config.copy()
                n[k] = v
                neighbors.append(n)
    return neighbors

def tabu_search_manager(target, meta_space, train_gen, val_gen, max_iters=2):
    print("\n" + "="*50)
    print(f"TABU SEARCH → {target}")
    print("="*50)

    current = {k: random.choice(v) for k, v in meta_space.items()}
    tabu = deque(maxlen=2)

    if target == "PSO":
        best_score = run_pso_worker(**current, train_gen=train_gen, val_gen=val_gen)
    else:
        best_score = run_sa_worker(**current, train_gen=train_gen, val_gen=val_gen)

    best_config = current.copy()

    for _ in range(max_iters):
        neighbors = get_neighbors(current, meta_space)
        best_neighbor = None
        best_neighbor_score = -1

        for n in neighbors:
            if tuple(n.items()) in tabu:
                continue

            if target == "PSO":
                score = run_pso_worker(**n, train_gen=train_gen, val_gen=val_gen)
            else:
                score = run_sa_worker(**n, train_gen=train_gen, val_gen=val_gen)

            if score > best_neighbor_score:
                best_neighbor_score = score
                best_neighbor = n

        if best_neighbor:
            current = best_neighbor
            tabu.append(tuple(current.items()))

            if best_neighbor_score > best_score:
                best_score = best_neighbor_score
                best_config = current.copy()

    return best_config, best_score



In [10]:
pso_meta_space = {
    "c1": [1.0, 1.5],
    "c2": [1.0, 1.5],
    "w":  [0.4, 0.6]
}

sa_meta_space = {
    "initial_temp": [1.0, 5.0],
    "cooling_rate": [0.8, 0.9]
}

best_pso, acc_pso = tabu_search_manager(
    "PSO", pso_meta_space, train_gen, val_gen
)



TABU SEARCH → PSO
    [Worker PSO] c1=1.0, c2=1.0, w=0.6


2025-12-19 11:36:59.664240: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


    [Worker PSO] c1=1.5, c2=1.0, w=0.6
    [Worker PSO] c1=1.0, c2=1.5, w=0.6
    [Worker PSO] c1=1.0, c2=1.0, w=0.4
    [Worker PSO] c1=1.5, c2=1.0, w=0.4
    [Worker PSO] c1=1.0, c2=1.5, w=0.4
    [Worker PSO] c1=1.0, c2=1.0, w=0.6


In [11]:
best_sa, acc_sa = tabu_search_manager(
    "SA", sa_meta_space, train_gen, val_gen
)

print("\nFINAL RESULTS")
print("TABU → PSO:", best_pso, acc_pso)
print("TABU → SA :", best_sa, acc_sa)



TABU SEARCH → SA
    [Worker SA] Temp=1.0, Cooling=0.9
    [Worker SA] Temp=5.0, Cooling=0.9
    [Worker SA] Temp=1.0, Cooling=0.8
    [Worker SA] Temp=1.0, Cooling=0.9
    [Worker SA] Temp=5.0, Cooling=0.8

FINAL RESULTS
TABU → PSO: {'c1': 1.0, 'c2': 1.0, 'w': 0.4} 0.48466256260871887
TABU → SA : {'initial_temp': 5.0, 'cooling_rate': 0.9} 0.46888694167137146
